# 20210119
- review image 데이터를 정답지(Y, N)으로 분류하여 별도 디렉토리에 저장

In [ ]:
# 주피터노트북 화면 넓게 보기 위한 용도
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
import boto3
import s3fs

import pymysql
from PIL import Image

from dotenv import load_dotenv
import os
load_dotenv() # .env 파일에서 정보를 갖고 오기 위함

In [ ]:
!pip install python-dotenv
# https://medium.com/@namseok.yoo/django-%EC%97%90%EC%84%9C-python-dotenv-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0-6d9b58a939ec

In [ ]:
conn = pymysql.connect(database=os.getenv('database'), user=os.getenv('user'), password=os.getenv('password'), host= 'localhost', port=3306)
qry = '''
    SELECT a.goods_no, a.est_no, b.image, a.point_yn
    FROM 
    (
        SELECT goods_no, no as est_no, point_yn
        FROM goods_estimate
        WHERE date_format(regi_date, '%Y%m%d') = '20201201'
        AND type = 'style'
    ) a
    JOIN
    (
        SELECT est_no, image
        FROM goods_estimate_image
        WHERE date_format(rt, '%Y%m%d') = '20201201'
        GROUP BY est_no, image HAVING count(img_no) = 1
    ) b
    ON a.est_no = b.est_no
'''
df = pd.read_sql_query(qry, conn)

In [ ]:
df['point_yn'].describe()

In [ ]:
df.head()

In [ ]:
def get_image(img_path):
    file = boto3.resource('s3').Object('musinsa', img_path[1:]).get()
    img = np.array(Image.open(file['Body']))
    return img

In [ ]:
# tf.keras.preprocessing.image_dataset_from_directory를 고려했으나 아래 아티클 참고하여 더 진행해야 할듯
# https://medium.com/analytics-vidhya/custom-keras-generator-fetching-images-from-s3-to-train-neural-network-4e98694de8ee
